In [ ]:
! pip install -U langgraph langsmith
! pip install langchain_community
! pip install langchain_openai

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END ,state
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

In [ ]:
 

# Configuration de la clé API
os.environ['OPENAI_API_KEY'] = "sk-proj-rPBnsXimJy JoLeEVAbOl0kiGIZTWPKAtav6nWikZPSewB3oeo055nFcLW9rnMorepgfeAZNQQz5FUUwrtmQsA"


# Création du modèle LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [6]:
step_back_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """
    Generate Step-Back Prompting based on user questions.
    
    Step-Back Prompting is a modification of Chain-of-Thought (CoT) where the LLM is first asked to consider high-level concepts or principles before delving into detailed reasoning.
    
    This technique significantly improves performance on complex reasoning tasks by encouraging the model to first understand the broader context and applicable principles before addressing specific details.
    
    The Step-Back process includes the following steps:
    1. High-level abstraction: First, identify the general principles, concepts, or frameworks that are relevant to the question.
    2. Conceptual analysis: Explore how these high-level concepts apply to the specific problem.
    3. Detailed reasoning: Once the conceptual framework is established, work through the specific problem step by step.
    4. Answer the original question: Provide a final answer based on the analysis.
    
    Please structure your response in the following format:
    
    Step 1: [High-level concepts and principles]
    Step 2: [Application of these concepts to the problem]
    Step 3: [Step-by-step reasoning]
    Step 4: [Final Answer]
    
    Example:
    Question: "A store is having a 25% off sale. If an item originally costs $80, what is the final price after the discount and 7% sales tax?"
    
    Step 1: High-level concepts and principles:
    Before diving into calculations, let me consider the key concepts involved in this problem:
    - Percentage discounts: When an item is discounted by X%, we multiply the price by (100%-X%) or (1-X/100)
    - Sales tax: Tax is applied after discounts, and we add X% tax by multiplying by (100%+X%) or (1+X/100)
    - Order of operations: In retail pricing, discounts are applied first, then tax is calculated on the discounted amount
    
    Step 2: Application of these concepts:
    For this specific problem:
    - We need to apply a 25% discount to the original price of $80
    - Then we need to apply a 7% sales tax to the discounted price
    
    Step 3: Step-by-step reasoning:
    1. Calculate the discounted price:
       - Original price = $80
       - Discount rate = 25% = 0.25
       - Discount amount = $80 × 0.25 = $20
       - Discounted price = $80 - $20 = $60
       - Alternatively: Discounted price = $80 × (1 - 0.25) = $80 × 0.75 = $60
    
    2. Calculate the tax on the discounted price:
       - Discounted price = $60
       - Tax rate = 7% = 0.07
       - Tax amount = $60 × 0.07 = $4.20
    
    3. Calculate the final price:
       - Final price = Discounted price + Tax amount
       - Final price = $60 + $4.20 = $64.20
    
    Step 4: The final price of the item after applying the 25% discount and 7% sales tax is $64.20.
    """),
    ("human", "{question}"),
    ("assistant", """
    Step 1: High-level concepts and principles:
    [Identify and explain the general principles, theories, or frameworks that are relevant to the question. What domain knowledge is necessary to understand this problem at a conceptual level?]
    
    Step 2: Application of these concepts:
    [Explain how these high-level concepts specifically relate to the problem at hand. What aspects of these principles are most relevant? How do they provide a framework for solving the specific question?]
    
    Step 3: Step-by-step reasoning:
    [Now that the conceptual groundwork is laid, work through the problem methodically:
    1. [First step in the solution process]
    2. [Second step in the solution process]
    3. [Continue with additional steps as needed]
    ...]
    
    Step 4: [Provide a clear, concise answer to the original question, drawing directly from the reasoning process above]
    """)
])

In [5]:
# Fonction pour générer le prompt ThoT (node)
def generate_thot_node(state):
    question = state['messages'][-1].content  # Récupère la dernière question
    prompt_value = step_back_prompt_template.invoke({"question": question})
    messages = prompt_value.to_messages()
    response = llm.invoke(messages)
    return {"messages": [response]}  # Ajoute la réponse comme un message

# Définition de l'état
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Création du graphe
graph_builder = StateGraph(State)
graph_builder.add_node("generate_thot", generate_thot_node)

# Configuration du graphe
graph_builder.set_entry_point("generate_thot")
graph_builder.add_edge("generate_thot", END)
graph = graph_builder.compile()

# Exemple d'utilisation
inputs = {"messages": [HumanMessage(content="Why can an increase in the minimum wage sometimes lead to higher unemployment, and under what conditions might this effect be minimized or reversed")]}

# Affiche la question posée au début
print("Question posée:", inputs["messages"][0].content)
print("\nRéponse:")

for output in graph.stream(inputs):
    for key, value in output.items():
        if key == "generate_thot":
            messages = value['messages']
            for message in messages:
                if isinstance(message, AIMessage):
                    print(message.content)

Question posée: Why can an increase in the minimum wage sometimes lead to higher unemployment, and under what conditions might this effect be minimized or reversed

Réponse:
Step 1: High-level concepts and principles:
- Labor Market Dynamics: The relationship between supply and demand for labor, where wages are determined by the equilibrium between employers' willingness to pay and workers' willingness to work.
- Minimum Wage Legislation: Government-imposed minimum wage laws that set the lowest legal wage that can be paid to workers.
- Elasticity of Labor Demand: The responsiveness of employers to changes in wage rates; if demand for labor is elastic, a rise in wages may lead to a significant reduction in the number of workers hired.
- Economic Conditions: The overall state of the economy, including factors like unemployment rates, inflation, and business cycles, which can influence the impact of minimum wage increases.

Step 2: Application of these concepts:
In this context, an increa